# Cleaning City_name

my target is categorize the city_name to fixed name for each city.


In [1]:
import pandas as pd
import numpy as np
import re
import json

In [2]:
data = pd.read_csv('./data/fixed/data_science_dataset_wuzzuf.csv')

In [16]:
# cleaning The "city_name" column 
# my target is to add each city to same name.
# print the unique city names entered.

# get the unique cities, print them to console + output them to a file 
unique_cities = np.unique(data.city_name)
print "** there are ", len(np.unique(data.city_name)), "unique city name"
with open('./data/generated/munging/unique_cities_unexpanded','wb') as f:
    for item in np.unique(data.city_name):
        f.write(item+"\n")

# Get each city with it's counts value, then extract the cities which has more than or equal three counts.
value_counts = data.city_name.value_counts()
unique_above_x = value_counts[value_counts>=3].keys()

#for the extracted cities , lower/strip them
unique_extracted = np.unique([ city.lower().strip() for city in unique_above_x ]) 
# then for the values which has more than city split them to a list, example "cairo and alex" > ['cairo','alex']
unique_extracted = [ re.split('[.,/&\-]|\sand\s|\sor\s',city,re.UNICODE) for city in unique_extracted ]

# convert the list of lists to a single list with all unique city names, and save them to a file
unique_list = [ s.strip() for item in unique_extracted for s in item  ]
unique_list = np.unique(unique_list)
with open('./data/generated/munging/unique_cities_expanded','wb') as f:
    for item in unique_list:
        f.write(item+'\n')

# add two more lists, one for english-typed city names, and the other for arabic-typed ones.
# patter to detect english characters
english_pattern = re.compile('^[\d\w\s]*$')
unique_english_list = []
unique_arabic_list = []
for city in unique_list:
    if english_pattern.match(city) :
        unique_english_list.append(city)
    else:
        unique_arabic_list.append(city)  

# print some stuff :)
print "** ", len(unique_list), " unique city names after filteration"
print "** ", len(unique_arabic_list), " Arabic city names"
print "** ", len(unique_english_list), " English city names"        

** there are  478 unique city name
**  104  unique city names after filteration
**  6  Arabic city names
**  98  English city names


In [18]:
# this json file includes standard city names as a key, and all the possible names as values.
with open('./data/fixed/cities_names_dict.json','rb') as f:
    cities_dict = json.load(f)

In [19]:
# this function takes a row, then change city name either to the standard name, or to 'other("name")'
# this done using the json file imported above.
def clean_city_name(d):
    city_name = d.city_name.lower().strip()
    cities = re.split('[.,/&\-]|\sand\s|\sor\s',city_name)
    new_cities = []
    for c in cities :
        if c == "": break;
        flag = True
        for item in cities_dict :
            for ii in cities_dict[item] :
                if re.search(ii, c) :
                    new_cities.append(item);
                    flag = False;
                    break
        if flag == True :
            new_cities.append("other('"+c.lower().strip()+"')")
    
    cities_str = ""
    for c in new_cities :
        if cities_str == "" : 
            cities_str += c
        else : 
            cities_str = cities_str +", "+c
    d.city_name = cities_str
    return d
                
new_data = data.apply(clean_city_name, axis=1)
# 297 others, and 21,533 checked

In [28]:
# now i want to expand the rows who have more than city_name to separted rows,
# example for city_name "cairo,alex" i want two rows on for cairo and one for alex.

# First i'll make a new dataframe "df2" to hold the rows I want to separate
#    and delete these rows from the main dataframe "new_data"
df2 = pd.DataFrame(columns=new_data.columns)
for x in new_data.iterrows() :
    index = x[0]
    row = x[1]
    if re.search(", ", row.city_name):
        df2.loc[len(df2)] = row
        new_data.drop(index, inplace=True)
# second part for the "df2" dataframe i'll expand the rows inside it.  
indexend = len(df2)
for x in df2.iterrows():
    index = x[0]
    row = x[1]
    cities = row.city_name.split(', ')
    for city in cities :
        alt = row
        alt.city_name = city
        df2.loc[indexend] = alt
        indexend += 1 
    df2.drop(index,inplace=True)
# third part is to concat this dataframe to the main dataframe "new_data",
# but before it lets reset the index for both of them
df2.reset_index(drop=True, inplace=True)
new_data.reset_index(drop=True, inplace=True)
index = len(new_data)
for x in df2.iterrows():
    row = x[1]
    new_data.loc[index] = row
    index += 1;

In [29]:
# this function takes a series [from our data] and format the date to %Y-%m ex: 2016-2
def format_date(r):
    r.post_date = re.findall("^\d{4}-\d{2}", r.post_date)[0]
    return r

In [30]:
# now I want to make a new dataframe to be the same as "new_data" but without the other("") cities.
data_cleaned = new_data.copy()
for x in data_cleaned.iterrows():
    index, row = x
    if re.search('other', row.city_name):
        data_cleaned.drop(index, inplace=True)
# reset it's index, format the year and save to a file.
data_cleaned.reset_index(drop=True, inplace=True)
data_cleaned = data_cleaned.apply(format_date, axis=1)
data_cleaned.to_csv('./data/generated/wazzuf/wuzzuf_data_cleaned.csv',index=False)

In [10]:
# here I save it but without the two columns "descrip, requirements" as they hold huge data.
data_cleaned_light = data_cleaned.drop(
    ["description",'job_requirements'], axis=1) 
data_cleaned_light.to_csv('./data/generated/wazzuf/wuzzuf_data_cleaned_light.csv', index=False)

In [11]:
# in case you want the data for some city/ies just put them in the following list and run this cell.
cities = [u'cairo', u'giza', u'alexandria', u'6th of october', u'10th of ramadan',
       u'mansoura', u'any']

data_with_some_cities = data_cleaned_light[ data_cleaned_light.city_name.isin(cities) ]

data_with_some_cities.to_csv('./data/generated/wazzuf/wuzzuf_data_with_some_cities.csv',index=False)

In [12]:
# save the cleaned data with date formated.
data_cleaned = data.apply(format_date, axis=1)
data_cleaned.to_csv('./data/generated/wazzuf/wuzzuf_data_formateddate.csv',index=False)

NameError: name 'format_date' is not defined

-----